Basic implementation of MLEnd Code

In [3]:
!pip install librosa

  Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
  Using cached soundfile-0.11.0-py2.py3-none-any.whl (23 kB)
  Using cached pooch-1.6.0-py3-none-any.whl (56 kB)
  Using cached audioread-3.0.0-py3-none-any.whl
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
#required dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa

import zipfile

In [5]:
directory_to_extract_to = '/home/jovyan/ML/DATA/DATA_unzip/'


zip_path = '/home/jovyan/ML/DATA/DATA_ZIP/MLEndLS_1.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [25]:
sample_path = '/home/jovyan/ML/DATA/DATA_unzip/*.wav'
files = glob.glob(sample_path)
len(files)

498

In [9]:
MLENDLS_df = pd.read_csv('/home/jovyan/ML/git-code/MLEndLS_final.csv').set_index('file_id') 
MLENDLS_df

,area,spot,in_out,Participant
file_id,,,,
0001.wav,british,street,outdoor,S151
0002.wav,kensington,dinosaur,indoor,S127
0003.wav,campus,square,outdoor,S18
0004.wav,kensington,hintze,indoor,S179
0005.wav,campus,square,outdoor,S176
...,...,...,...,...
2496.wav,westend,trafalgar,outdoor,S151
2497.wav,campus,square,outdoor,S6
2498.wav,westend,national,indoor,S96


In [12]:
n=0
fs = None
x, fs = librosa.load(files[n],sr=fs)
print('This audio signal has', len(x), 'samples')

This audio signal has 422919 samples


In [13]:
def getPitch(x,fs,winLen=0.02):
  #winLen = 0.02 
  p = winLen*fs
  frame_length = int(2**int(p-1).bit_length())
  hop_length = frame_length//2
  f0, voiced_flag, voiced_probs = librosa.pyin(y=x, fmin=80, fmax=450, sr=fs,
                                                 frame_length=frame_length,hop_length=hop_length)
  return f0,voiced_flag

In [14]:
def getXy(files,labels_file, scale_audio=False, onlySingleDigit=False):
  X,y =[],[]
  for file in tqdm(files):
    fileID = file.split('/')[-1]
    file_name = file.split('/')[-1]
    yi = labels_file.loc[fileID]['in_out']=='indoor'

    fs = None # if None, fs would be 22050
    x, fs = librosa.load(file,sr=fs)
    if scale_audio: x = x/np.max(np.abs(x))
    f0, voiced_flag = getPitch(x,fs,winLen=0.02)
      
    power = np.sum(x**2)/len(x)
    pitch_mean = np.nanmean(f0) if np.mean(np.isnan(f0))<1 else 0
    pitch_std  = np.nanstd(f0) if np.mean(np.isnan(f0))<1 else 0
    voiced_fr = np.mean(voiced_flag)

    xi = [power,pitch_mean,pitch_std,voiced_fr]
    X.append(xi)
    y.append(yi)

  return np.array(X),np.array(y)

In [23]:
files[27]

'/home/jovyan/ML/DATA/DATA_unzip/0021.wav'

In [26]:
X,y = getXy(files, labels_file=MLENDLS_df, scale_audio=True, onlySingleDigit=True)

100%|██████████| 498/498 [08:12<00:00,  1.01it/s]


In [27]:
print('The shape of X is', X.shape) 
print('The shape of y is', y.shape)
print('The labels vector is', y)

The shape of X is (498, 4)
The shape of y is (498,)
The labels vector is [False False  True  True False False False False  True False  True False
 False False False False False  True  True False False False False False
 False False  True  True False False False False False False  True  True
  True  True  True False False  True  True  True  True False  True False
  True  True False  True  True False False False  True False  True False
 False False False False  True  True False  True  True False  True  True
  True False  True  True False False False False  True False False  True
 False  True False  True False  True False False False False  True False
 False  True False  True False False False False False  True False  True
 False False  True False  True  True False False False  True False False
 False False False False False  True False False  True False  True  True
  True False  True  True  True  True False False False  True False False
 False False False False False  True False  True Fa

In [28]:
print(' The number of indoor recordings is ', np.count_nonzero(y))
print(' The number of outdoor recordings is ', y.size - np.count_nonzero(y))

 The number of indoor recordings is  210
 The number of outdoor recordings is  288


In [29]:
from sklearn import svm
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((348, 4), (150, 4), (348,), (150,))

In [30]:
model  = svm.SVC(C=1)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))
print('The support vectors are', model.support_vectors_.shape)

Training Accuracy 0.5718390804597702
Validation  Accuracy 0.5933333333333334
The support vectors are (306, 4)


In [31]:
mean = X_train.mean(0)
sd =  X_train.std(0)

X_train = (X_train-mean)/sd
X_val  = (X_val-mean)/sd

model  = svm.SVC(C=1,gamma=2)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))
print('The support vectors are', model.support_vectors_.shape)

Training Accuracy 0.75
Validation  Accuracy 0.5466666666666666
The support vectors are (323, 4)
